# 💻 Digital Access in Public Libraries (FY 2022)

In [3]:
import pandas as pd

# Load raw IMLS AE dataset (includes EBOOK, ELMATCIR, POPU_UND)
df_raw = pd.read_csv("PLS_FY22_AE_pud22i.csv", encoding="cp1252")

This notebook analyzes digital access across U.S. public libraries using data from the FY 2022 IMLS Public Libraries Survey.

We explore:
- 📚 E-book collections  
- 🔁 Electronic materials circulation  
- 👥 Digital access normalized by population


📄 All data from the IMLS Public Libraries Survey (FY 2022).

## 🔁 Electronic Circulation per 1,000 People

In [5]:
# Re-aggregate digital ebook metrics at the state level
state_digital = df_raw.groupby('STABR', as_index=False).agg({
    'EBOOK': 'sum',
    'POPU_UND': 'sum'
})
state_digital['Ebooks per 1k'] = state_digital['EBOOK'] / (state_digital['POPU_UND'] / 1000)
state_digital = state_digital.rename(columns={'STABR': 'State'})

# Cap at 95th percentile
threshold = state_digital['Ebooks per 1k'].quantile(0.95)
state_digital['Ebooks per 1k Capped'] = state_digital['Ebooks per 1k'].clip(upper=threshold)

# Plot
import plotly.express as px
fig = px.choropleth(
    state_digital,
    locations='State',
    locationmode='USA-states',
    color='Ebooks per 1k Capped',
    color_continuous_scale='Greens',
    scope='usa',
    title='Ebooks per 1,000 People by State (Capped at 95th Percentile)',
    labels={'Ebooks per 1k Capped': 'Ebooks / 1k residents (capped)'}
)
fig.show()

In [7]:
# Aggregate electronic circulation by state
state_circ = df_raw.groupby('STABR', as_index=False).agg({
    'ELMATCIR': 'sum',
    'POPU_UND': 'sum'
})
state_circ['E-Circ per 1k'] = state_circ['ELMATCIR'] / (state_circ['POPU_UND'] / 1000)
state_circ = state_circ.rename(columns={'STABR': 'State'})

# Cap at 95th percentile
ecirc_threshold = state_circ['E-Circ per 1k'].quantile(0.95)
state_circ['E-Circ per 1k Capped'] = state_circ['E-Circ per 1k'].clip(upper=ecirc_threshold)

# Plot capped E-Circ choropleth
import plotly.express as px
fig = px.choropleth(
    state_circ,
    locations='State',
    locationmode='USA-states',
    color='E-Circ per 1k Capped',
    color_continuous_scale='Teal',
    scope='usa',
    title='Electronic Circulation per 1,000 People by State (Capped at 95th Percentile)',
    labels={'E-Circ per 1k Capped': 'E-Circ / 1k residents (capped)'}
)
fig.show()